# 5.1 UNIVARIATE PORTFOLIO ANALYSIS

## 5.1.7 Analyzing Returns

In [1]:
# -*- coding:utf-8 -*-
import pandas as pd
import numpy as np
from scipy import stats
import os
import datetime
import statsmodels.formula.api as smf
from dateutil.parser import parse
import statsmodels.api as sm
os.chdir(r'E:\python3\Empirial_Asset_Pricing')
path = os.getcwd()

In [2]:
# 为什么要读取月度因子数据并计算年度的因子值？
# 因为回归是用年度数据

# 读取月度因子数据
factor_monthly = pd.read_csv('data/F-F_Year_Factors.csv')
factor_monthly['dateff'] = factor_monthly['dateff'].apply(int).apply(str).apply(parse)
factor_monthly = factor_monthly[(factor_monthly['dateff'] >= datetime.datetime(1987, 12, 31, 0, 0))&(factor_monthly['dateff'] <= datetime.datetime(2013, 1, 1, 0, 0))]
factor_monthly.rename(columns = {'dateff':'month','mktrf':'mkt'},inplace = True)
factor_monthly = factor_monthly[['month','mkt','smb','hml','umd']]

# 由于常用因子值也为根据变量值分组投资组合的收益率，因此将月度数据按年度连乘(+1后)计算年度因子值
factor_yearly = factor_monthly.copy()
factor_yearly['year'] = factor_yearly['month'].apply(lambda x:x.year)
factor_yearly[['mkt','smb','hml','umd']] = factor_yearly[['mkt','smb','hml','umd']].applymap(lambda x:x+1)
factor_yearly = factor_yearly.groupby('year').prod()
factor_yearly = factor_yearly.applymap(lambda x:(x-1)*100)
factor_yearly = factor_yearly.reset_index()

# 由于个股数据表中中汇报的收益率为t+1期，故此处年份减1使因子值也为t+1期，便于后面连接
factor_yearly_yearlag1 = factor_yearly.copy()
factor_yearly_yearlag1['year'] = factor_yearly_yearlag1['year']-1

In [31]:
# factor_monthly
# factor_yearly

单变量组合分析流程

In [4]:
# 读取个股数据
all_data = pd.read_csv(os.path.join(path,'data','alldata2021.csv'),index_col=0)
all_data = all_data.reset_index()
all_data = all_data.drop_duplicates(subset=['PERMNO','year'])
all_data = all_data[['PERMNO', 'year', 'beta', 'rt_1', 'BM','MktCap']]
all_data['rt_1'] = all_data['rt_1']*100

# 分别提取不同变量所需分组数据至多个dataframe
proxy_name_list = ['beta','MktCap','BM']
def select_proxy(proxy_name_list):
    for proxy_name in proxy_name_list:
        globals()[proxy_name] = all_data[['PERMNO', 'year','rt_1',proxy_name]]
select_proxy(proxy_name_list)

## 做等权重的7分组
# 根据bm,beta,mktcap分别进行单变量7分组
def mutate_group(proxy_name_list):
    for proxy_name in proxy_name_list:
        globals()[proxy_name] = globals()[proxy_name].dropna()

        quantiles_proxy = globals()[proxy_name].groupby('year')[proxy_name].describe(
        percentiles=[0.1,0.2,0.4,0.6,0.8,0.9]).reset_index()[['year','10%','20%','40%','60%','80%','90%']]

        df= pd.merge(globals()[proxy_name], quantiles_proxy, how = 'left', on = 'year')

        globals()[proxy_name]['group'] = np.select([df[proxy_name] <= df['10%'],
                           (df[proxy_name] > df['10%']) & (df[proxy_name] <= df['20%']),
                           (df[proxy_name] > df['20%']) & (df[proxy_name] <= df['40%']),
                            (df[proxy_name] > df['40%']) & (df[proxy_name] <= df['60%']),
                            (df[proxy_name] > df['60%']) & (df[proxy_name] <= df['80%']),
                            (df[proxy_name] > df['80%']) & (df[proxy_name] <= df['90%']),
                           (df[proxy_name] > df['90%'])],
                           ['1','2','3','4','5','6','7'])
mutate_group(proxy_name_list)

# 计算等权投资组合收益率
def portfolio_ret(proxy_name_list):
    for proxy_name in proxy_name_list:
        globals()[proxy_name] = globals()[proxy_name].dropna()

        globals()[proxy_name] = globals()[proxy_name].groupby(['group', 'year']).apply(
            lambda x: np.average(x['rt_1'], weights=None)).reset_index()
        globals()[proxy_name].rename(columns={0: 'ret_excess'}, inplace=True)
        globals()[proxy_name] = pd.pivot(globals()[proxy_name], index='year', columns='group')[
            'ret_excess'].reset_index()

        # 没有股票的组用0填充,此算例中数据分布基本均匀且无明显缺失，应用不到
        globals()[proxy_name].replace(np.nan,0,inplace = True)

        globals()[proxy_name]['7-1'] = globals()[proxy_name]['7'] - globals()[proxy_name]['1']
portfolio_ret(proxy_name_list)

# 定义NW调整函数
def nw_adjust(df, group, lags=6):
    df.dropna(subset = [group], inplace = True)
    adj_a = np.array(df[group])
    # 对常数回归
    model = sm.OLS(adj_a, [1] * len(adj_a)).fit(cov_type='HAC', cov_kwds={'maxlags': lags})
    return adj_a.mean().round(2), format(float(model.tvalues), ".2f"),float(model.bse) #第三个参数为系数标准差，此处用不上

In [5]:
beta

group,year,1,2,3,4,5,6,7,7-1
0,1988,-0.870104,1.246955,2.509111,7.022783,3.654271,9.243924,8.839346,9.709450
1,1989,-30.246377,-28.610393,-28.609753,-29.856064,-27.677079,-26.557099,-25.702105,4.544272
2,1990,57.940546,27.440364,36.963121,40.539133,55.580769,65.686471,66.610264,8.669717
3,1991,55.759940,31.693029,30.843056,22.218697,19.325835,16.010873,20.028960,-35.730979
4,1992,36.729256,28.415352,23.411731,24.297349,20.183382,15.052093,8.093602,-28.635654
5,1993,-4.172467,-4.573412,-5.307025,-3.919032,-5.841288,-10.588274,-4.660616,-0.488149
6,1994,28.932509,21.977124,25.303196,29.937316,27.180291,24.069666,37.405417,8.472908
7,1995,21.491805,16.432629,19.033217,14.382417,12.569086,11.778138,9.125073,-12.366732
8,1996,23.448517,40.597391,29.004326,20.847228,17.888905,6.854548,-8.037257,-31.485773
9,1997,-6.662887,-8.843913,-6.894050,-10.050289,-6.572323,-6.730838,0.377691,7.040578


TABLE 5.8: Average Returns of Portfolios Sorted on 𝜷, MktCap, and BM

In [6]:
# generate table8
table8_index = ['Beta','','MKtCap','','BM','']
table8_columns = ['1','2','3','4','5','6','7','7-1']
table8 = pd.DataFrame(index = table8_index, columns = table8_columns)
table8.columns.name = 'Sort Variable'
def generate_table8(proxy_name_list):
    for i in range(len(proxy_name_list)):
        for j in range(len(table8_columns)):
            table8.iloc[2*i,j] = nw_adjust(globals()[proxy_name_list[i]], table8_columns[j])[0]
            table8.iloc[2*i+1,j] = nw_adjust(globals()[proxy_name_list[i]],table8_columns[j])[1]
generate_table8(proxy_name_list)
table8

Sort Variable,1,2,3,4,5,6,7,7-1
Beta,16.73,14.09,14.82,12.91,12.22,11,10.52,-6.21
,4.62,5.09,5.17,5.65,5.36,4.66,3.02,-1.44
MKtCap,29.11,16.54,12.33,10.41,8.69,9.28,8.33,-20.78
,6.58,4.68,3.93,4.42,4.86,5.14,4.02,-5.07
BM,5.48,7.16,9.84,11.21,15.02,19.03,25.87,20.39
,1.87,3.03,4.80,5.30,6.68,6.68,6.87,8.37


TABLE 5.9: 𝜷-Sorted Portfolio Risk-Adjusted Results

Capital Asset Pricing Model (CAPM) of Sharpe (1964), Lintner (1965), and Mossin (1966)

$r_{p, t}=\alpha+\beta_{M K T} M K T_{t}+\epsilon_{t}$

Fama and French (FF) three-factor model,Fama and French (1993)

$r_{p, t}=\alpha+\beta_{M K T} M K T_{t}+\beta_{S M B} S M B_{t}+\beta_{H M L} H M L_{t}+\epsilon_{t}$

Fama, French, and Carhart (FFC) four-factor model,Jegadeesh and Titman (1993) and Carhart (1997)

$\begin{aligned} r_{p, t}=& \alpha+\beta_{M K T} M K T_{t}+\beta_{S M B} S M B_{t} +\beta_{H M L} H M L_{t}+\beta_{M O M} M O M_{t}+\epsilon_{t} \end{aligned}$

In [7]:
# generate table9
table9_index = ['Excess return','',
                'CAPM','','','',
                'FF','','','','','','','',
                'FFC','','','','','','','','','']
table9_columns = ['Coefficient','1','2','3','4','5','6','7','7-1']
table9 = pd.DataFrame(index = table9_index, columns = table9_columns)
table9.columns.name = 'Model'
table9['Coefficient'] = ['Excess return','',
                         'alpha','','MKT','',
                         'alpha','','MKT','','SMB','','HML','',
                         'alpha','','MKT','','SMB','','HML','','MOM','']

# 连接因子模型因子数据
# 起初在根据年份连接时未如前所述在因子表中将年份减一，导致实际用t期因子值解释t+1期收益，无解释力，
# 因子模型解释后alpha反变大，现已修正
def mutate_factor(proxy_name_list):
    for proxy_name in proxy_name_list:
        globals()[proxy_name] = pd.merge(globals()[proxy_name], factor_yearly_yearlag1, how='left', on=['year'])
mutate_factor(proxy_name_list)

# 定义用于风险调整的CAPM,FF3,FFC因子模型函数
def capm_adjust(df, group, lags = 6):
    df.dropna(subset = [group,'mkt'], inplace = True)
    x = sm.add_constant(df['mkt'])
    y = df[group]
    model = sm.OLS(y, x).fit(cov_type='HAC', cov_kwds={'maxlags': lags})
    return model

def ff3_adjust(df, group, lags = 6):
    df.dropna(subset=[group, 'mkt', 'smb', 'hml'], inplace = True)
    x = sm.add_constant(df[['mkt', 'smb', 'hml']])
    y = df[group]
    model = sm.OLS(y, x).fit(cov_type='HAC', cov_kwds={'maxlags': lags})
    return model

def ffc_adjust(df, group, lags = 6):
    df.dropna(subset=[group, 'mkt', 'smb', 'hml', 'umd'], inplace = True)
    x = sm.add_constant(df[['mkt', 'smb', 'hml','umd']])
    y = df[group]
    model = sm.OLS(y, x).fit(cov_type='HAC', cov_kwds={'maxlags': lags})
    return model

# 分别计算不同因子模型调整后的风险收益及t值
def generate_table9():
    for j in range(len(table9.columns) - 1):
        table9.iloc[0,j+1] = nw_adjust(globals()['beta'], table9_columns[j+1])[0]
        table9.iloc[1,j+1] = nw_adjust(globals()['beta'], table9_columns[j+1])[1]

        for i in range(2):
            table9.iloc[2+2*i, j+1] = capm_adjust(globals()['beta'], table9_columns[j+1]).params[i].round(2)
            table9.iloc[3+2*i, j+1] = capm_adjust(globals()['beta'], table9_columns[j+1]).tvalues[i].round(2)

        for i in range(4):
            table9.iloc[6+2*i, j+1] = ff3_adjust(globals()['beta'], table9_columns[j+1]).params[i].round(2)
            table9.iloc[7+2*i, j+1] = ff3_adjust(globals()['beta'], table9_columns[j+1]).tvalues[i].round(2)

        for i in range(5):
            table9.iloc[14+2*i, j+1] = ffc_adjust(globals()['beta'], table9_columns[j+1]).params[i].round(2)
            table9.iloc[15+2*i, j+1] = ffc_adjust(globals()['beta'], table9_columns[j+1]).tvalues[i].round(2)
generate_table9()
table9

Model,Coefficient,1,2,3,4,5,6,7,7-1
Excess return,Excess return,16.73,14.09,14.82,12.91,12.22,11,10.52,-6.21
,,4.62,5.09,5.17,5.65,5.36,4.66,3.02,-1.44
CAPM,alpha,9.12,6.95,6.99,5.45,3.83,0.5,-2.37,-11.5
,,1.93,1.56,1.61,1.5,1.27,0.22,-1.09,-1.96
,MKT,1.03,0.96,1.06,1.01,1.13,1.42,1.74,0.71
,,3.63,3.45,4.26,4.28,5.7,8.18,8.74,1.99
FF,alpha,4.09,2.12,2.78,1.85,1.25,-0.65,-1.25,-5.34
,,1.76,1.59,2.63,2.13,1.05,-0.41,-0.53,-1.7
,MKT,1.11,1.07,1.12,1.06,1.13,1.34,1.56,0.44
,,9.05,6.81,9.44,10.61,14.98,13.9,11.35,2.09


# 5.2 BIVARIATE INDEPENDENT-SORT ANALYSIS

## 5.2.1 Breakpoints

Letting $n_{P 1}$ represent the number of groups that will be created based on the firs sort variable and $n_{P 2}$ be the number of groups that will be created based on the second sort variable, the number of portfolios that will be formed is $n_{P 1} \times n_{P 2}$. There are therefore $n_{P 1}$ and $n_{P 2}$ breakpoints for the first and second sort variables, respectively.

The breakpoints for each of the two sort variables are calculated in exactly the same way as for a univariate portfolio analysis. The breakpoints (percentiles used to calculate the breakpoints) used to form the groups for the first sort variable are denoted $B 1_{j, t}\left(p 1_{j}\right)$ for $j \in\left\{1,2, \ldots, n_{P 1}-1\right\}$, and the breakpoints (percentiles) for the second sort variable are $B 2_{k, t}\left(p 2_{k}\right)$ for $k \in\left\{1,2, \ldots, n_{P 2}-1\right\}$. The actual breakpoints are calculated as

$B 1_{j, t}=\operatorname{Pctl}_{p 1_{j}}\left(\left\{X 1_{t}\right\}\right)$

$B 2_{k, t}=\operatorname{Pctl}_{p 2_{k}}\left(\left\{X 2_{t}\right\}\right)$

where $\operatorname{Pctl}_{p}(Z)$ is the $p$ th percentile of the set $Z$ and $\left\{X 1_{t}\right\}$ and $\left\{X 2_{t}\right\}$ are the set of available values of $X 1$ and $X 2$, respectively, in period $t$.

TABLE 5.10: Bivariate Independent-Sort Breakpoints

In [9]:
# generate table10
# bivariate independent-sort portfolio，
# The first sort variable is beta and the second sort variable is MktCap；
def generate_table10():
    quantiles_proxy_beta = all_data.groupby('year')['beta'].describe(
            percentiles=[0.3, 0.7]).reset_index()[['year', '30%', '70%']]

    quantiles_proxy_mktcap = all_data.groupby('year')['MktCap'].describe(
            percentiles=[0.25, 0.5, 0.75]).reset_index()[['year', '25%', '50%', '75%']]

    beta_mktcap_group = pd.merge(quantiles_proxy_beta, quantiles_proxy_mktcap, on=['year'])
    beta_mktcap_group.rename(columns = {'30%':'B1_1t','70%':'B1_2t',
                                        '25%':'B2_1t','50%':'B2_2t','75%':'B2_3t'},inplace=True)
    return beta_mktcap_group
table10 = generate_table10()
table10.iloc[:,1:] = table10.iloc[:,1:].applymap(lambda x:round(x,2))
table10

,year,B1_1t,B1_2t,B2_1t,B2_2t,B2_3t
0,1988,0.18,0.66,9.24,32.86,147.05
1,1989,0.17,0.69,9.55,35.24,172.07
2,1990,0.23,0.86,6.34,24.47,137.00
3,1991,0.24,0.85,10.14,41.99,212.18
4,1992,0.25,0.97,16.38,61.59,268.27
5,1993,0.28,0.92,22.25,73.19,299.14
6,1994,0.36,0.97,20.14,67.32,272.35
7,1995,0.27,0.90,25.75,89.00,355.28
8,1996,0.32,0.90,28.50,97.32,403.62
9,1997,0.25,0.72,32.89,115.42,480.37


## 5.2.2 Portfolio Formation

As with the univariate portfolio analysis, the next step in bivariate portfolio analysis is to form the periodic portfolios. As mentioned earlier, if there are $n_{P 1}$ groups based on the first sort variable $X 1$ and $n_{P 2}$ groups based on the second sort variable $X 2$, then there will be $n_{P 1} \times n_{P 2}$ portfolios each time period. The portfolios for period $t$ are denoted $P_{j, k, t}$, where the first subscript indicates the group of the first sort variable and the second subscript indicates that of the second sort variable. In general, the portfolios are defined as

$P_{j, k, t}=\left\{i \mid B 1_{j-1, t} \leq X 1_{i, t} \leq B 1_{j, t}\right\} \cap\left\{i \mid B 2_{k-1, t} \leq X 2_{i, t} \leq B 2_{k, t}\right\}$

for $j \in\left\{1,2, \ldots, n_{P 1}\right\}, \quad k \in\left\{1,2, \ldots, n_{P 2}\right\}, \quad$ where $\quad B 1_{0, t}=B 2_{0, t}=-\infty$,
$B 1_{n_{p 1}, t}=B 2_{n_{p 2, t}}=\infty$, and $\cap$ is the intersection operator. Thus, for a given entity $i$ to be held in portfolio $P_{j, k, t}$, the entity must have a value of $X 1$ in period $t$ that is between the $j-1$ st and $j$ th (inclusive) period $t$ breakpoints for the first sort variable, and also have a period $t$ value of $X 2$ between the $k-1$ st and $k$ th (inclusive) period $t$ breakpoints for the second sort variable.

TABLE 5.11: Bivariate Independent-Sort Number of Stocks per Portfolio

In [11]:
# generate table11
all_data_and_breakpoints = pd.merge(all_data,table10,on='year')
def portfolio_ind_3x4():
    df = all_data_and_breakpoints.copy()
    df = df.dropna(subset=['beta','MktCap'])
    df['X1_group'] = np.select([
        (df['beta'] < df['B1_1t']),
        (df['beta'] >= df['B1_1t']) & (df['beta'] < df['B1_2t']),
        (df['beta'] >= df['B1_2t'])],
        ['1','2','3'])
    df['X2_group'] = np.select([
        (df['MktCap'] < df['B2_1t']),
        (df['MktCap'] >= df['B2_1t']) & (df['MktCap'] < df['B2_2t']),
        (df['MktCap'] >= df['B2_2t']) & (df['MktCap'] < df['B2_3t']),
        (df['MktCap'] >= df['B2_3t'])],
        ['1','2','3','4'])
    return df[['PERMNO','year','beta','rt_1','MktCap','X1_group','X2_group']]
all_data_and_groups = portfolio_ind_3x4()

def generate_table11(group_table):
    n = group_table.groupby(['year', 'X1_group', 'X2_group'])['PERMNO'].count().reset_index().rename(
            columns={'PERMNO': 'n_firms'})
    df = pd.pivot_table(n, index=['year', 'X2_group'], columns='X1_group')['n_firms'].reset_index()
    df.rename(columns={'1':'beta1','2':'beta2','3':'beta3'},inplace=True)
    return df
table11 = generate_table11(all_data_and_groups)
table11

X1_group,year,X2_group,beta1,beta2,beta3
0,1988,1,731,466,216
1,1988,2,541,577,287
2,1988,3,344,696,407
3,1988,4,106,572,814
4,1989,1,726,423,231
...,...,...,...,...,...
95,2011,4,177,540,223
96,2012,1,663,150,72
97,2012,2,138,390,357
98,2012,3,79,447,361


![title](table12.png)

TABLE 5.13: Bivariate Independent-Sort Portfolio Excess Returns

In [13]:
# generate table13
def avg_r(group_table):
    ewret = group_table.groupby(['year','X2_group','X1_group'])['rt_1'].mean().reset_index()
    df = pd.pivot_table(ewret,index=['year','X2_group'],columns='X1_group')['rt_1'].reset_index()
    df['diff'] = df['3']-df['1']
    df['avg'] = (df['1']+df['2']+df['3'])/3
    df.loc[:,'1':'avg']= df.loc[:,'1':'avg'].applymap(lambda x:round(x,2))
    return df
table13 = avg_r(all_data_and_groups)

def get_diff(table):
    X = pd.DataFrame()
    for i in range(1988,2012):
        x = table[table['year']==i]
        x1 = pd.DataFrame(columns=x.columns,index=list(range(2)))
        x1['year'] = [i,i]
        x1.iloc[0,1] = ['mktcap_diff']
        x1.iloc[1,1] = ['mktcap_avg']
        x1.iloc[0, 2:] = (x.iloc[3, 2:] - x.iloc[0, 2:]).apply(lambda x:round(x,2))
        x1.iloc[1, 2:] = x.iloc[0:3, 2:].mean().apply(lambda x:round(x,2))
        X = pd.concat([X, x])
        X = pd.concat([X, x1])
    return X
table13 = get_diff(table13)
table13 = table13.reset_index(drop= True)
table13

X1_group,year,X2_group,1,2,3,diff,avg
0,1988,1,1.18,-2.38,2.62,1.43,0.47
1,1988,2,-1.02,0.29,-11.21,-10.19,-3.98
2,1988,3,4.11,3.54,4.87,0.77,4.17
3,1988,4,9.62,14.28,17.26,7.65,13.72
4,1988,mktcap_diff,8.44,16.66,14.64,6.22,13.25
...,...,...,...,...,...,...,...
139,2011,2,28.8,20.62,19.14,-9.66,22.85
140,2011,3,11.22,16.88,19.92,8.7,16.01
141,2011,4,11.84,17.8,17.53,5.69,15.73
142,2011,mktcap_diff,-16.19,6.41,27.88,44.07,6.04


TABLE 5.14: Bivariate Independent-Sort Portfolio Excess and Abnormal Returns

In [14]:
# generate table14
##计算综合excess return和FFC调整alpha
table14_sub_index = ['Excess return','','FFC alpha','']
table14_sub_column = ['beta1','beta2','beta3','beta Diff','beta Avg']
table14_sub = pd.DataFrame(index=table14_sub_index,columns=table14_sub_column)
subs = table13['X2_group'].unique()

def generate_table14_subs():
    for sub in subs:
        globals()[sub + '_table'] = table14_sub.copy()
        globals()[sub + '_data'] = table13[table13['X2_group']==sub].copy()
        globals()[sub + '_data'].loc[:,'1':'avg'] = globals()[sub + '_data'].loc[:,'1':'avg'].applymap(lambda x:np.float64(x))
        globals()[sub + '_data'] = pd.merge(globals()[sub + '_data'],factor_yearly_yearlag1,on='year')
        for i in range(len(globals()[sub + '_table'].columns)):
            globals()[sub + '_table'].iloc[0,i] = nw_adjust(globals()[sub + '_data'],globals()[sub + '_data'].columns[2+i])[0]
            globals()[sub + '_table'].iloc[1,i] = nw_adjust(globals()[sub + '_data'],globals()[sub + '_data'].columns[2+i])[1]
            globals()[sub + '_table'].iloc[2,i] = ffc_adjust(globals()[sub + '_data'],globals()[sub + '_data'].columns[2+i]).params[0].round(2)
            globals()[sub + '_table'].iloc[3,i] = ffc_adjust(globals()[sub + '_data'],globals()[sub + '_data'].columns[2+i]).tvalues[0].round(2)
generate_table14_subs()

table14 = pd.DataFrame()
for sub in subs:
    table14 = pd.concat([table14, globals()[sub + '_table']])
table14 = table14.reset_index().reset_index()
table14['level_0'] = ['MktCap 1','','','',
                'MktCap 2', '', '', '',
                'MktCap 3', '', '', '',
                'MktCap 4', '', '', '',
                'MktCap Diff', '', '', '',
                'MktCap Avg', '', '', '',]
table14.rename(columns={'index':'Coefficient',
                        'level_0':''},inplace=True)
table14

,,Coefficient,beta1,beta2,beta3,beta Diff,beta Avg
0,MktCap 1,Excess return,21.08,23.81,20.83,-0.25,21.91
1,,,5.66,5.59,3.33,-0.06,5.08
2,,FFC alpha,12.85,18.6,9.01,-3.84,13.49
3,,,5.04,6.37,1.29,-0.71,3.59
4,MktCap 2,Excess return,11.45,13.9,11.99,0.54,12.45
5,,,3.97,3.73,3.79,0.17,4.35
6,,FFC alpha,0.76,2.25,1.61,0.86,1.54
7,,,0.39,0.75,0.68,0.22,1.13
8,MktCap 3,Excess return,9.16,10.64,8.71,-0.45,9.5
9,,,3.02,4.75,3.92,-0.13,4.63
